In [30]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder

In [31]:
import xgboost
from xgboost import XGBClassifier #xgboost==0.90
xgboost.__version__

'1.7.4'

In [32]:
data = pd.read_csv('../Field_training_data_noOutliers_newContaminationMetric.csv')
labels = pd.read_csv('../Field_training_labels_noOutliers_newContaminationMetric.csv')
data = data.reset_index(drop=True)
labels = labels.reset_index(drop=True)

idx = labels.index[labels['Trophic mode'] == 'Un']
train_labels = labels.drop(idx)
train_data = data.drop(idx)

features = pd.read_csv('../Extracted_Pfams_noOutliers_newContaminationMetric_xg.csv')

g2Inc_TPM_merged_lats = pd.read_csv('../../g2Incubation/g2Inc_tpm_updatedMarferret_marmicroDb2023_noOutliers_fixedTPM_fall2023.csv')

,PF00003,PF00013,PF00022,PF00034,PF00071,PF00124,PF00125,PF00139,PF00169,PF00185,...,PF14259,PF14727,PF14778,PF14782,PF14870,PF15104,PF15782,PF16158,PF16313,PF16672
0,0.000000,292.332516,401.432567,25.495096,4274.120745,0.0,825.784894,0,927.875223,175.291985,...,0,244.526371,0,0.000000,51.772177,0.000000,0,0.000000,0.000000,0.000000
1,0.000000,374.843479,167.069213,0.000000,455.787826,0.0,0.000000,0,78.197677,5.501400,...,0,0.000000,0,0.000000,116.214036,0.000000,0,0.000000,0.000000,0.000000
2,31.124248,339.465432,546.664798,1199.697343,3319.753802,0.0,146.841898,0,327.899683,0.000000,...,0,0.000000,0,0.000000,340.409396,0.000000,0,7.556099,0.000000,0.000000
3,0.000000,4.215522,2331.394112,109.886334,3401.692168,0.0,296.223252,0,405.546821,0.000000,...,0,9.567227,0,18.050046,0.000000,27.933804,0,51.798278,0.000000,0.000000
4,0.000000,381.725778,1437.911184,520.035168,1763.542029,0.0,394.370870,0,477.495308,61.877529,...,0,175.666892,0,14.661634,177.471135,42.098196,0,0.000000,15.667544,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,0.000000,406.212040,179.223067,11.338150,604.302083,0.0,289.916434,0,279.120211,61.086612,...,0,0.000000,0,6.609195,3.894012,15.785731,0,4.197296,18.364944,0.000000
251,0.000000,0.000000,3476.550156,567.485015,5795.064492,0.0,267.201175,0,612.274666,0.000000,...,0,0.000000,0,0.000000,0.000000,42.460201,0,11.879384,0.000000,0.000000
252,0.000000,447.000258,276.136569,0.000000,713.448463,0.0,167.392168,0,22.067612,39.184697,...,0,0.000000,0,6.150229,12.154696,25.161991,0,53.692722,29.667616,61.452074
253,0.000000,332.731410,305.765498,80.408262,870.429051,0.0,213.101154,0,415.370753,35.576400,...,0,5.360098,0,26.957157,35.417666,25.289330,0,3.424595,31.555905,0.000000


In [33]:
g2Inc_TPM_merged_lats.columns[(g2Inc_TPM_merged_lats == 0).all()]

Index(['PF00139', 'PF00349', 'PF05033', 'PF05427', 'PF07784', 'PF08713',
       'PF08839', 'PF14259', 'PF14778', 'PF15782'],
      dtype='object')

In [35]:
features = features['pfam']

In [42]:
len(features)

183

In [43]:
g2Inc_TPM_merged_lats = g2Inc_TPM_merged_lats[features]

In [45]:
g2Inc_TPM_merged_lats.columns[(g2Inc_TPM_merged_lats == 0).all()]

Index(['PF05427', 'PF14259', 'PF00139', 'PF05033', 'PF14778', 'PF08839',
       'PF08713', 'PF00349', 'PF15782', 'PF07784'],
      dtype='object')

In [46]:
train_data = train_data[features]

In [47]:
le = LabelEncoder()

In [48]:
X,y = train_data, le.fit_transform(train_labels['Trophic mode'])
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [49]:
g2Inc = scaler.transform(g2Inc_TPM_merged_lats)

In [50]:
#gamma: 0.5, learning_rate: 0.2, max_depth: 3, n_estimators: 100, reg_lambda: 0.0 
model = XGBClassifier(gamma=0.5, learning_rate=0.2, max_depth=3, n_estimators=100, reg_lambda=0.0)
model.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.5, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [51]:
xg_predictions_g2Inc = model.predict(g2Inc)

In [53]:
prob = model.predict_proba(g2Inc)

In [54]:
g2Inc_predictions = pd.DataFrame(data={'xg_pred':xg_predictions_g2Inc})

In [55]:
probDF = pd.DataFrame(data={'probability':np.max(prob, axis = 1)})

In [57]:
g2Inc_predictions.to_csv('../../g2Incubation/g2Inc_trophicModePredictions_updatedMarferret_marmicroDb2023_noOutliers_fall2023',index=False)

In [58]:
probDF.to_csv('../../g2Incubation/g2Inc_trophicModePredictionsProbabilities_updatedMarferret_marmicroDb2023_noOutliers_fall2023',index=False)